In [0]:
with cte_data (x, y, z, p, cnt) as
(
select split_part(value, ',', 1)::int as x
     , split_part(value, ',', 2)::int as y
     , split_part(value, ',', 3)::int as z
     , (x * 1e10 + y * 1e5 + z)::bigint
     , count(*) over()
  from read_files('/Volumes/waldar/fabien/aoc2025/aoc2025_08_input.txt', format => 'text', wholeText => false) as t (value)
/*from values ('162,817,812')
            , ('57,618,57')
            , ('906,360,560')
            , ('592,479,940')
            , ('352,342,300')
            , ('466,668,158')
            , ('542,29,236')
            , ('431,825,988')
            , ('739,650,466')
            , ('52,470,668')
            , ('216,146,977')
            , ('819,987,18')
            , ('117,168,530')
            , ('805,96,715')
            , ('346,949,466')
            , ('970,615,88')
            , ('941,993,340')
            , ('862,61,35')
            , ('984,92,344')
            , ('425,690,689') as t (value)*/
)
  ,  cte_matrix (p1, p2, rn, cnt) as
(
 select t1.p
      , t2.p
      , row_number() over W as rn
      , t1.cnt
   from cte_data as t1
   join cte_data as t2
  where t1.p < t2.p
 window W as (order by power(t1.x - t2.x, 2) + power(t1.y - t2.y, 2) + power(t1.z - t2.z, 2) asc)
)
select aggregate( slice(array_sort(array_agg(struct(rn, p1, p2))), 1, case any_value(cnt) when 20 then 10 else 1000 end)
                , array()::array<array<bigint>>
                , (circuits, m) -> case array_size(filter(circuits, e -> array_contains(e, m.p1) or array_contains(e, m.p2)))
                                     when 0
                                     then array_append(circuits, array(m.p1, m.p2))
                                     when 1
                                     then array_sort(transform(filter(circuits, e -> array_contains(e, m.p1) or array_contains(e, m.p2)), t -> array_distinct(t || array(m.p1, m.p2)))
                                       || filter(circuits, e -> not array_contains(e, m.p1) and not array_contains(e, m.p2)))
                                     else array_sort(array(flatten(filter(circuits, e -> array_contains(e, m.p1) or array_contains(e, m.p2))))
                                       || filter(circuits, e -> not array_contains(e, m.p1) and not array_contains(e, m.p2)))
                                   end
               , circuits -> aggregate(slice(sort_array(transform(circuits, c -> array_size(c)), false), 1, 3), 1l, (acc, x) -> acc * x)
               ) as part1
     , aggregate( array_sort(array_agg(struct(rn, p1, p2)))
                , struct(0l as xx, array()::array<array<bigint>> as c)
                , (circuits, m) -> case
                                     when size(circuits.c) = 1
                                      and size(circuits.c[0]) = any_value(cnt)
                                     then circuits
                                     else struct( (m.p1 div 1e10::bigint) * (m.p2 div 1e10::bigint) as xx
                                                , case array_size(filter(circuits.c, e -> array_contains(e, m.p1) or array_contains(e, m.p2)))
                                                     when 0
                                                     then array_append(circuits.c, array(m.p1, m.p2))
                                                     when 1
                                                     then array_sort(transform(filter(circuits.c, e -> array_contains(e, m.p1) or array_contains(e, m.p2)), t -> array_distinct(t || array(m.p1, m.p2)))
                                                       || filter(circuits.c, e -> not array_contains(e, m.p1) and not array_contains(e, m.p2)))
                                                     else array_sort(array(flatten(filter(circuits.c, e -> array_contains(e, m.p1) or array_contains(e, m.p2))))
                                                       || filter(circuits.c, e -> not array_contains(e, m.p1) and not array_contains(e, m.p2)))
                                                  end as c)
                                   end
               , circuits -> circuits.xx
               ) as part2
  from cte_matrix
;






  ,  cte_recurs (circuits, rn) max recursion level 1001 as
(
select array(array(p1, p2))
     , rn
  from cte_matrix
 where rn = 1
 union all
select case array_size(filter(r.circuits, e -> array_contains(e, m.p1) or array_contains(e, m.p2)))
         when 0
         then array_append(r.circuits, array(m.p1, m.p2))
         when 1
         then transform(filter(r.circuits, e -> array_contains(e, m.p1) or array_contains(e, m.p2)), t -> array_distinct(t || array(m.p1, m.p2)))
           || filter(r.circuits, e -> not array_contains(e, m.p1) and not array_contains(e, m.p2))
         else array(flatten(filter(r.circuits, e -> array_contains(e, m.p1) or array_contains(e, m.p2))))
           || filter(r.circuits, e -> not array_contains(e, m.p1) and not array_contains(e, m.p2))
       end
     , m.rn
  from cte_recurs as r
  join cte_matrix as m on m.rn = r.rn + 1
)
select aggregate(slice(sort_array(transform(max_by(circuits, rn), c -> array_size(c)), false), 1, 3), 1l, (acc, x) -> acc * x) as part1
  from cte_recurs
;

